In [ ]:
from PIL import Image
import numpy as np
import torch
from fastai.vision.models import Darknet

from gradcam.misc_functions import get_example_params, save_class_activation_images

In [ ]:
class CamExtractor():
    """
        Extracts cam features from the model
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None

    def save_gradient(self, grad):
        self.gradients = grad

    def forward_pass_on_convolutions(self, x):
        """
            Does a forward pass on convolutions, hooks the function at given layer
        """
        conv_output = None
#         for module_pos, module in self.model.features._modules.items():
        for module_pos, module in enumerate(m.module.layers):
            x = module(x)  # Forward
            if int(module_pos) == self.target_layer:
                x.register_hook(self.save_gradient)
                conv_output = x  # Save the convolution output on that layer
        return conv_output, x

    def forward_pass(self, x):
        """
            Does a full forward pass on the model
        """
        # Forward pass on the convolutions
        conv_output, x = self.forward_pass_on_convolutions(x)
#         x = x.view(x.size(0), -1)  # Flatten
#         # Forward pass on the classifier
#         x = self.model.classifier(x)
        return conv_output, x

In [ ]:
class GradCam():
    """
        Produces class activation map
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.model.eval()
        # Define extractor
        self.extractor = CamExtractor(self.model, target_layer)

    def generate_cam(self, input_image, target_class=None):
        # Full forward pass
        # conv_output is the output of convolutions at specified layer
        # model_output is the final output of the model (1, 1000)
        conv_output, model_output = self.extractor.forward_pass(input_image)
        if target_class is None:
            target_class = np.argmax(model_output.data.numpy())
        # Target for backprop
        one_hot_output = torch.FloatTensor(1, model_output.size()[-1]).zero_()
        one_hot_output[0][target_class] = 1
        # Zero grads
#         self.model.features.zero_grad()
#         self.model.classifier.zero_grad()
        # Backward pass with specified target
        model_output.backward(gradient=one_hot_output, retain_graph=True)
        # Get hooked gradients
        guided_gradients = self.extractor.gradients.data.numpy()[0]
        # Get convolution outputs
        target = conv_output.data.numpy()[0]
        # Get weights from gradients
        weights = np.mean(guided_gradients, axis=(1, 2))  # Take averages for each gradient
        # Create empty numpy array for cam
        cam = np.ones(target.shape[1:], dtype=np.float32)
        # Multiply each weight with its conv output and then, sum
        for i, w in enumerate(weights):
            cam += w * target[i, :, :]
        cam = np.maximum(cam, 0)
        cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))  # Normalize between 0-1
        cam = np.uint8(cam * 255)  # Scale between 0-255 to visualize
        cam = np.uint8(Image.fromarray(cam).resize((input_image.shape[2],
                       input_image.shape[3]), Image.ANTIALIAS))
        # ^ I am extremely unhappy with this line. Originally resizing was done in cv2 which
        # supports resizing numpy matrices, however, when I moved the repository to PIL, this
        # option is out of the window. So, in order to use resizing with ANTIALIAS feature of PIL,
        # I briefly convert matrix to PIL image and then back.
        # If there is a more beautiful way, send a PR.
        return cam

In [ ]:

# Get params
target_example = 1  # Lato


In [ ]:
def preprocess_image(pil_im, resize_im=True):
    """
        Processes image for CNNs

    Args:
        PIL_img (PIL_img): Image to process
        resize_im (bool): Resize to 224 or not
    returns:
        im_as_var (torch variable): Variable that contains processed float tensor
    """
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    # Resize image
    if resize_im:
        pil_im.thumbnail((512, 512))
    im_as_arr = np.float32(pil_im)
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = torch.autograd.Variable(im_as_ten, requires_grad=True)
    return im_as_var

In [ ]:


example_list = (('/home/paperspace/code/fontastic/data/stratified/train/Lora/Lora-Regular_300_rand_crop_8.jpg', 2),
                    ('/home/paperspace/code/fontastic/data/stratified/train/Lato/Lato-Regular_300_rand_crop_6.jpg', 1),
                )
example_index = 0

img_path = example_list[example_index][0]
target_class = example_list[example_index][1]
file_name_to_export = img_path[img_path.rfind('/')+1:img_path.rfind('.')]
# Read image
original_image = Image.open(img_path).convert('RGB')
# Process image
prep_img = preprocess_image(original_image)
# Define model



In [ ]:
import torch.nn as nn

In [ ]:
m = Darknet([1, 2, 4, 6, 3], num_classes=6, nf=16)
m = nn.DataParallel(m, device_ids=None)

m.load_state_dict(torch.load('/home/paperspace/code/fontastic/data/stratified/darknet.pth'))
#     return (original_image,
#             prep_img,
#             target_class,
#             file_name_to_export,
#             pretrained_model)

In [ ]:
m = m.to(torch.device("cpu"))

In [ ]:
# Grad cam
grad_cam = GradCam(m, target_layer=20)


In [ ]:
device = torch.device("cpu")

prep_img.to(device)

In [ ]:
# Generate cam mask
cam = grad_cam.generate_cam(prep_img, target_class)
# Save mask


In [ ]:
f'{file_name_to_export}-gradcam-darknet'

In [ ]:
im = Image.fromarray(cam)

In [ ]:
save_class_activation_images(original_image, cam, f'{file_name_to_export}-gradcam-darknet')
print('Grad cam completed')

In [ ]:
Image.open('../results/Lora-Regular_300_rand_crop_8-gradcam-darknet_Cam_On_Image.png')